<a href="https://colab.research.google.com/github/LakeRainSound/text_mining_practice2/blob/master/LDA_TA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# nltkの文書群にgensimのLDAを適用してみる

## 準備

必要ライブラリ(全部pipで入ります)
 * nltk
 * gensim
 * pyLDAvis

In [1]:
!pip install nltk
!pip install gensim
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 1.3MB/s 
     |████████████████████████████████| 552kB 2.1MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=d42108e59439bd04fd7d4c5bf6573de13965f2c8f4b84eb9b229ebd693d5a29e
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=4faa5992d4ce36463a9157a9b51db2999c2ea6a6e105d3f9bb5a2e5c0fe1fa12
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [2]:
#nltk使ったことない人は、pip install してから対話環境等で以下のdataset等をダウンロードしてください
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("reuters")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## データロード・前処理

In [3]:
#dataset読み込み
from nltk.corpus import reuters as corpus

### 今回はこういう文書(をBOW化したもの)を用います

In [4]:
for n,item in enumerate(corpus.words(corpus.fileids()[0])[:300]):
    print(item, end=" ")
    if (n%25) ==24:
      print(" ")

ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears  
among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said . They  
told Reuter correspondents in Asian capitals a U . S . Move against Japan might boost protectionist sentiment in the U . S . And  
lead to curbs on American imports of their products . But some exporters said that while the conflict would hurt them in the long -  
run , in the short - term Tokyo ' s loss might be their gain . The U . S . Has said it will  
impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to  
stick to a pact not to sell semiconductors on world markets at below cost . Unofficial Japanese estimates put the impact of the tariffs at  
10 billion dlrs and spokesmen for major electronics firms said they would virtually halt exports 

In [5]:
#全document数
len(corpus.fileids())

10788

In [6]:

#前からk個のdocumentのみで学習する場合
#k=1000
#docs=[corpus.words(fileid) for fileid in corpus.fileids()[:k]]

#全documentで学習する場合
docs=[corpus.words(fileid) for fileid in corpus.fileids()]

print(docs[:5])
print("num of docs:", len(docs))

[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', ...], ['CHINA', 'DAILY', 'SAYS', 'VERMIN', 'EAT', '7', '-', ...], ['JAPAN', 'TO', 'REVISE', 'LONG', '-', 'TERM', ...], ['THAI', 'TRADE', 'DEFICIT', 'WIDENS', 'IN', 'FIRST', ...], ['INDONESIA', 'SEES', 'CPO', 'PRICE', 'RISING', ...]]
num of docs: 10788


## 前処理 

In [7]:
#ストップワードリストの作成

#1 nltkのストップワードリスト
en_stop = nltk.corpus.stopwords.words('english')

# 一度LDAしてみる等して，適宜ノイズになってそうな記号等を見つけて，ストップワードリストに新たに加える
#【発展】記号や数字は正規表現で消してみましょう
en_stop= ["``","/",",.",".,",";","--",":",")","(",'"','&',"'",'),',',"','-','.,','.,"','.-',"?",">","<"]                  \
         +["0","1","2","3","4","5","6","7","8","9","10","11","12","86","1986","1987","000"]                                                      \
         +["said","say","u","v","mln","ct","net","dlrs","tonne","pct","shr","nil","company","lt","share","year","billion","price"]          \
         +en_stop

In [8]:
#前処理関数の作成

from nltk.corpus import wordnet as wn #lemmatize関数のためのimport

def preprocess_word(word, stopwordset):
    
    #1.make words lower  example: Python =>python
    word=word.lower()
    
    #2.remove "," and "."
    if word in [",","."]:
        return None
    
    #3.remove stopword  example: the => (None) 
    if word in stopwordset:
        return None
    
    #4.lemmatize  example: cooked=>cook
    lemma = wn.morphy(word)
    if lemma is None:
        return word

    elif lemma in stopwordset: #lemmatizeしたものがstopwordである可能性がある
        return None
    else:
        return lemma
    

def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

In [9]:
#before
print(docs[0][:25]) 

#after
print(preprocess_documents(docs)[0][:25])

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears']
['asian', 'exporter', 'fear', 'damage', 'japan', 'rift', 'mounting', 'trade', 'friction', 'japan', 'raise', 'fear', 'among', 'many', 'asia', 'exporting', 'nation', 'row', 'could', 'inflict', 'far', 'reaching', 'economic', 'damage', 'businessmen']


## LDA準備

In [10]:
import gensim
from gensim import corpora

In [11]:
#documentを，gensim LDAが読み込めるデータ構造にする

#辞書の作成
dictionary = corpora.Dictionary(preprocess_documents(docs))
#コーパスの作成
corpus_ = [dictionary.doc2bow(doc) for doc in preprocess_documents(docs)]

In [12]:
#Dictionary:gensimにおける辞書クラス
#token2id属性には単語と辞書IDとの対応が格納される

print(dictionary.token2id)

{'15': 0, '17': 1, '1985': 2, '30': 3, '300': 4, '53': 5, '95': 6, '>.': 7, 'able': 8, 'accounting': 9, 'action': 10, 'advantage': 11, 'allege': 12, 'allow': 13, 'also': 14, 'american': 15, 'among': 16, 'analyst': 17, 'april': 18, 'asia': 19, 'asian': 20, 'ask': 21, 'association': 22, 'australia': 23, 'australian': 24, 'avow': 25, 'await': 26, 'aware': 27, 'barrier': 28, 'beef': 29, 'beyond': 30, 'big': 31, 'block': 32, 'boost': 33, 'broker': 34, 'budget': 35, 'business': 36, 'businessmen': 37, 'button': 38, 'call': 39, 'canberra': 40, 'capel': 41, 'capital': 42, 'centre': 43, 'chairman': 44, 'chief': 45, 'co': 46, 'coal': 47, 'commercial': 48, 'complete': 49, 'concern': 50, 'conflict': 51, 'continue': 52, 'correspondent': 53, 'cost': 54, 'could': 55, 'country': 56, 'curb': 57, 'cut': 58, 'damage': 59, 'day': 60, 'defuse': 61, 'democratic': 62, 'deputy': 63, 'despite': 64, 'deterioration': 65, 'diplomatic': 66, 'director': 67, 'disadvantage': 68, 'dispute': 69, 'domestically': 70, 'due

In [13]:
#corpusにはdocumentごとに単語の(ID、出現回数)のリストが得られる

print(corpus_[0][:10]) #文章での出現順でなく辞書IDの若い順なことに注意

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 3), (8, 1), (9, 1)]


In [14]:
#before
print([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]])

#after
print(dictionary.doc2bow([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]]))

#これを全文書の全文に適用したのがcorpus_

['asian', 'exporters', 'fear', 'damage', 'from', 'u', '.', 's', '.-', 'japan', 'rift', 'mounting', 'trade', 'friction', 'between', 'the', 'u', '.', 's', '.', 'and', 'japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.']
[(16, 1), (19, 1), (20, 1), (37, 1), (55, 1), (59, 2), (72, 1), (85, 1), (88, 1), (89, 1), (96, 1), (116, 1), (120, 2), (142, 1), (157, 1), (198, 1), (209, 1), (210, 1), (256, 1)]


## LDA学習
### k=6のとき

In [15]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=6,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )

## パラメータの確認

In [16]:
#(トピックID, 当該トピックにおける単語とそのprobability)  ※　のうち、上位num_words位 

topics = ldamodel.print_topics(num_words=15)
for topic in topics:
    print(topic)

(0, '0.011*"record" + 0.010*"april" + 0.009*"dividend" + 0.008*"stock" + 0.008*"pay" + 0.008*"inc" + 0.007*"corp" + 0.007*"expect" + 0.007*"quarter" + 0.006*"div" + 0.006*"prior" + 0.006*"qtly" + 0.006*"set" + 0.006*"one" + 0.006*"may"')
(1, '0.018*"bank" + 0.011*"market" + 0.010*"january" + 0.010*"february" + 0.008*"rise" + 0.007*"stg" + 0.007*"dollar" + 0.007*"last" + 0.007*"rose" + 0.007*"month" + 0.006*"money" + 0.006*"rate" + 0.005*"export" + 0.005*"banks" + 0.005*"week"')
(2, '0.009*"corp" + 0.008*"offer" + 0.007*"co" + 0.007*"inc" + 0.007*"ltd" + 0.006*"group" + 0.006*"stake" + 0.006*"buy" + 0.006*"sell" + 0.005*"would" + 0.005*"unit" + 0.005*"one" + 0.005*"oil" + 0.005*"new" + 0.005*"50"')
(3, '0.015*"trade" + 0.010*"would" + 0.010*"japan" + 0.007*"market" + 0.007*"agreement" + 0.006*"dollar" + 0.006*"exchange" + 0.006*"currency" + 0.006*"."" + 0.005*"tell" + 0.005*"country" + 0.005*"japanese" + 0.005*"ec" + 0.005*"also" + 0.004*"baker"')
(4, '0.037*"loss" + 0.021*"profit" + 0.

In [17]:
#[(当該documentにおけるトピックIDとそのprobability　)]　 ※　のうち、minimum_probabilityの値を超えるもの

for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(1, 0.047957283), (2, 0.05130565), (3, 0.6753122), (5, 0.22493881)]
document ID 1:[(4, 0.084687315), (5, 0.90811723)]
document ID 2:[(1, 0.24314299), (2, 0.29777858), (3, 0.24455297), (5, 0.2127167)]
document ID 3:[(1, 0.45358065), (4, 0.25132874), (5, 0.29185015)]
document ID 4:[(1, 0.31460792), (3, 0.15512751), (4, 0.029980216), (5, 0.4982557)]
document ID 5:[(1, 0.14972213), (3, 0.073543616), (5, 0.7740456)]
document ID 6:[(1, 0.25547206), (3, 0.1974709), (4, 0.012895576), (5, 0.5333124)]
document ID 7:[(2, 0.5495651), (5, 0.43736103)]
document ID 8:[(2, 0.99270993)]
document ID 9:[(0, 0.28146565), (1, 0.36594513), (2, 0.07197263), (3, 0.24022342), (5, 0.040121812)]


In [18]:
#documentのcategory
categories=[corpus.categories(fileid) for fileid in corpus.fileids()]

In [19]:
n=0

#n番目のdocumentのトピック分布
print(ldamodel.get_document_topics(corpus_[n]))

#n番目のdocumentのcategory
print(categories[n])

#n番目のdocumentの生の文章
print(" ".join(docs[n]))

[(1, 0.047955953), (2, 0.051301587), (3, 0.6753014), (5, 0.2249551)]
['trade']
ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said . They told Reuter correspondents in Asian capitals a U . S . Move against Japan might boost protectionist sentiment in the U . S . And lead to curbs on American imports of their products . But some exporters said that while the conflict would hurt them in the long - run , in the short - term Tokyo ' s loss might be their gain . The U . S . Has said it will impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to stick to a pact not to sell semiconductors on world markets at below cost . Unofficial Japanese estimates put the impact of the tariffs at 10 billion dlrs and spokesmen fo

## 可視化

In [20]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [21]:
#全documentを学習に用いた場合結構時間がかかる(20min~)
#gensimではK個のトピックに0~K-1のidが割り振られていたのに対し，pyLDAvisでは1~Kのidが割り振られていることに注意

lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [22]:
#上で可視化したモデルをgoogle drive上にsaveできる

pyLDAvis.save_html(lda_display,'vis6.html')

In [23]:
!ls

sample_data  vis.html


## 別のkでの結果

### k=4のとき

In [24]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=4,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )


/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

### パラメータの確認

In [25]:
#(トピックID, 当該トピックにおける単語とそのprobability)  ※　のうち、上位num_words位 

topics = ldamodel.print_topics(num_words=15)
for topic in topics:
    print(topic)

(0, '0.012*"inc" + 0.012*"corp" + 0.011*"stock" + 0.009*"offer" + 0.007*"group" + 0.006*"co" + 0.006*"common" + 0.006*"shareholder" + 0.005*"would" + 0.005*"buy" + 0.005*"acquire" + 0.005*"acquisition" + 0.005*"one" + 0.005*"unit" + 0.005*"oil"')
(1, '0.009*"would" + 0.009*"trade" + 0.008*"market" + 0.007*"oil" + 0.007*"bank" + 0.006*"last" + 0.005*"export" + 0.005*"dollar" + 0.005*"japan" + 0.005*"government" + 0.004*"official" + 0.004*"country" + 0.004*"foreign" + 0.004*"also" + 0.004*".""')
(2, '0.010*"stg" + 0.009*"january" + 0.008*"february" + 0.006*"market" + 0.006*"bank" + 0.005*"last" + 0.004*"k" + 0.004*"increase" + 0.004*"rise" + 0.004*"rose" + 0.004*"corn" + 0.004*"new" + 0.004*"department" + 0.004*"expect" + 0.004*"total"')
(3, '0.034*"loss" + 0.020*"profit" + 0.015*"qtr" + 0.014*"rev" + 0.010*"inc" + 0.010*"note" + 0.009*"sales" + 0.009*"31" + 0.008*"oper" + 0.008*"1985" + 0.008*"include" + 0.007*"quarter" + 0.007*"record" + 0.007*"4th" + 0.006*"corp"')


### 可視化

In [26]:
#全documentを学習に用いた場合結構時間がかかる(20min~)
#gensimではK個のトピックに0~K-1のidが割り振られていたのに対し，pyLDAvisでは1~Kのidが割り振られていることに注意

lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)